# Support Vector Machine, SVM
-  Support Vector Machine (SVM) is a supervised machine learning algorithm primarily used for classification tasks, although it can also be adapted for regression. The main idea of SVM is to find the optimal hyperplane that best separates the classes in the dataset. The hyperplane is chosen in such a way that the margin between the two classes is maximized, which means SVM tries to find the boundary that leaves the largest possible distance between different classes. This helps in improving the model's ability to generalize on unseen data.

- SVM works well in high-dimensional spaces and is particularly effective in cases where the number of dimensions exceeds the number of data points. It also utilizes kernel functions to transform non-linearly separable data into a higher-dimensional space, making it easier to classify.

# How is SVM Used in Our Project?
- In the context of customer churn prediction, SVM can be used to classify whether a customer is likely to churn or stay. The goal of our project is to predict customer behavior based on a variety of customer-related features. By training an SVM classifier on historical data (which contains information about customer behaviors and whether they churned), the model will learn patterns that help distinguish churned customers from non-churned customers.

For example:

- Feature Selection: We provide SVM with features such as tenure, monthly charges, service type, etc. These features help SVM understand which factors are most influential in determining whether a customer will churn.
- Kernel Trick: If the relationship between features and churn is not linearly separable, we can use kernel functions (e.g., radial basis function or polynomial kernel) to map these features to a higher-dimensional space where a linear boundary may become more apparent.
- Classification: The SVM model is then used to create a decision boundary that separates customers who are likely to churn from those who are not. Once trained, this decision boundary can be used to predict the likelihood of churn for new or unseen customers.

In [1]:
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd

df = pd.read_csv("../2_data/telcocustomerchurn_featured.csv")
# Define the features and target variable
X = df.drop(columns=[ 'Churn'])
y = df['Churn']


# StratifiedShuffleSplit 
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=7)
print(sss)

for train_index, test_index in sss.split(X, y):
    print("train:", train_index, "test:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

print(f"X shape: {X.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y shape: {y.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Save the train and test splits to CSV files
X_train.to_csv("../2_data/X_train.csv", index=False)
X_test.to_csv("../2_data/X_test.csv", index=False)
y_train.to_csv("../2_data/y_train.csv", index=False)
y_test.to_csv("../2_data/y_test.csv", index=False)

StratifiedShuffleSplit(n_splits=5, random_state=7, test_size=0.2,
            train_size=None)
train: [4486 1855 3910 ... 4687  690 5658] test: [ 724  945 4710 ... 3730 1633 2496]
train: [ 937 6833 3567 ... 4257 4716 5657] test: [6171  505 5903 ... 6911 2257 5571]
train: [4793 2570 3406 ...  805 2103 1162] test: [  31 2128 2079 ... 2313 6356 5365]
train: [6799  180 3061 ... 3155 2428  793] test: [3083 4441  980 ... 4552 2903 1479]
train: [ 603 4034 6970 ... 1356 4927 1489] test: [5931 4868 2690 ... 3344 6206 5245]
X shape: (7043, 115)
X_train shape: (5634, 115)
X_test shape: (1409, 115)
y shape: (7043,)
y_train shape: (5634,)
y_test shape: (1409,)


- In this step, we are using Support Vector Machine (SVM) to classify whether customers will churn or not. First, we import the SVC model from sklearn.svm, then create an instance of SVM with the RBF kernel. We fit the model using the training data (X_train, y_train) and make predictions (y_pred) on the test data (X_test).

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import recall_score, precision_score
import pandas as pd

Classifiers = [('Support Vector Machine', SVC(class_weight='balanced'))]

Classify_result = []
names = []
prediction = []

for name, classifier in Classifiers:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    print("y_test distribution:", pd.Series(y_test).value_counts())
    print("y_pred distribution:", pd.Series(y_pred).value_counts())

    recall = recall_score(y_test, y_pred, zero_division=1)
    precision = precision_score(y_test, y_pred, zero_division=1)

    class_eva = pd.DataFrame([{'Recall': recall, 'Precision': precision}])
    Classify_result.append(class_eva)

    name = pd.Series(name)
    names.append(name)

    y_pred = pd.Series(y_pred)
    prediction.append(y_pred)


y_test distribution: 0    1035
1     374
Name: Churn, dtype: int64
y_pred distribution: 0    737
1    672
dtype: int64


# Evaluation Metrics
## Accuracy
- In this code snippet, the accuracy of the Support Vector Machine (SVM) model on the test dataset is calculated. The accuracy metric indicates the proportion of correctly classified instances among the total instances. In this case, the accuracy score is 0.78, meaning that the model is able to correctly classify 78% of the test samples.

- While an accuracy of 78% may seem acceptable, it's essential to keep in mind that for a customer churn prediction problem, we usually deal with imbalanced data. Relying solely on accuracy might not give a complete picture of the model's performance, as the model might be biased towards the majority class (e.g., predicting that most customers will not churn). It's crucial to evaluate other metrics like precision, recall, F1-score, and ROC-AUC to gain more insights into the model's effectiveness, especially in identifying the minority (churned) class.

In [5]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.53


## Precision and Recall
- We calculate two evaluation metrics: precision and recall for the Support Vector Machine (SVM) model. These metrics provide insights into how well the model is predicting the churned customers, which is important for customer churn prediction projects.

- Precision (0.73): Precision represents the proportion of correctly predicted churned customers (true positives) out of all customers predicted to churn (true positives + false positives). In this case, precision is 0.73, which means that 73% of the customers that the model predicted to churn actually did churn. This metric tells us how precise our model is in identifying actual churned customers.

- Recall (0.30): Recall represents the proportion of actual churned customers (true positives) that the model was able to identify out of all churned customers (true positives + false negatives). In this case, recall is 0.30, which means that the model is able to identify only 30% of the actual churned customers. This indicates that the model has a relatively low sensitivity and may be missing many churned customers.

- In customer churn prediction projects, having a high recall is usually crucial to ensure that as many churned customers as possible are correctly identified so that retention strategies can be applied.

In [6]:
from sklearn.metrics import precision_score, recall_score

# Compute Recall and Precision
precision = precision_score(y_test, y_pred, pos_label=1)  # label 1 as churn
recall = recall_score(y_test, y_pred, pos_label=1)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

Precision: 0.28
Recall: 0.51


## F1-Score
- In this code snippet, the F1-Score is calculated for the predictions made by the SVM classifier.

- F1-Score (0.43): The F1-Score is the harmonic mean of precision and recall, providing a single metric that balances both. It is particularly useful when dealing with imbalanced datasets, as it accounts for both false positives and false negatives. An F1-Score of 0.43 means that the model's performance in terms of precision and recall is relatively low. The low F1-Score indicates that although the model has some ability to correctly predict churned customers, there is still significant room for improvement, especially regarding balancing the identification and accuracy of churned customers.

In [7]:
from sklearn.metrics import f1_score

# compute F1-Score
f1 = f1_score(y_test, y_pred, pos_label=1)
print(f'F1-Score: {f1:.2f}')


F1-Score: 0.36


## ROC-AUC Score
-  the ROC-AUC Score is calculated to evaluate the performance of the SVM classifier.

- ROC-AUC Score (0.63): The ROC-AUC Score (Receiver Operating Characteristic - Area Under the Curve) measures the model's ability to distinguish between positive and negative classes. A score of 1.0 represents perfect classification, while 0.5 represents random guessing. In this example, the ROC-AUC score is 0.63, which is only slightly better than random, indicating that the model has limited capability in distinguishing between churned and non-churned customers. The model requires further tuning or improvement to achieve better classification performance.

In [8]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, y_pred)
print(f'ROC-AUC Score: {roc_auc:.2f}')


ROC-AUC Score: 0.52


# Summary
- The performance of the SVM model in customer churn prediction is not satisfactory, particularly in terms of recall and distinguishing capability (ROC-AUC). The low recall and F1-score indicate that the model has missed a significant number of churned customers, which could negatively impact business decisions.
